In [1]:
import ctypes
import numpy as np

In [2]:
intrinsic_gemm_so = ctypes.CDLL('./libintrinsic_gemm.so')
print(dir(intrinsic_gemm_so))

['_FuncPtr', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_func_flags_', '_func_restype_', '_handle', '_name']


In [6]:
get_instance_func = intrinsic_gemm_so.get_instance
get_instance_func.argtypes = None
get_instance_func.restype = ctypes.c_void_p

instance_init_func = intrinsic_gemm_so.instance_init
instance_init_func.argtypes = (ctypes.c_void_p, ctypes.c_bool, ctypes.c_bool, ctypes.c_int, ctypes.c_int, ctypes.c_int)
instance_init_func.restype = ctypes.c_int

instance_dispatch_func = intrinsic_gemm_so.instance_dispatch
instance_dispatch_func.argtypes =  (ctypes.c_void_p, ctypes.c_float, ctypes.c_float, ctypes.c_char_p, ctypes.c_char_p, ctypes.POINTER(ctypes.c_int))
instance_dispatch_func.restype = ctypes.c_int

In [9]:
m, n, k = 2, 2, 3
matrix_a = np.arange(0, m*k, 1, dtype=np.int8)
matrix_b = np.arange(0, n*k, 1, dtype=np.int8)
matrix_c = np.arange(0, m*n, 1, dtype=np.int32)

intrinnsic_gemm_engine = get_instance_func()
status = instance_init_func(intrinnsic_gemm_engine, False, False, m, n, k)
status = instance_dispatch_func(intrinnsic_gemm_engine, 1.0, 1.0, matrix_a.ctypes.data_as(ctypes.c_char_p),
                                                                    matrix_b.ctypes.data_as(ctypes.c_char_p), matrix_c.ctypes.data_as(ctypes.POINTER(ctypes.c_int)))

In [10]:
for arg in matrix_c:
    print(arg)

5
14
14
50
